Às vezes, queremos selecionar dados com base em grupos e entender dados agregados em nível de grupo. Vimos que, embora o Pandas nos permita iterar em cada linha em um dataframe, geralmente é muito lento fazê-lo. Felizmente, o Pandas tem uma função groupby() para acelerar essa tarefa. A ideia por trás da função groupby() é que ela pega algum dataframe, divide-o em pedaços com base em alguns valores de chave, aplica computação nesses pedaços e depois combina os resultados novamente em outro dataframe. Nos pandas, isso é chamado de padrão split-apply-combine.

# Splitting

In [3]:
import pandas as pd
import numpy as np

#vamos trazer o census data
df = pd.read_csv('/content/census.csv')
# e manter somente os dados com SUMLEV == 50
df = df[df['SUMLEV']==50]
df.head()




,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [4]:

# No primeiro exemplo para groupby() eu quero usar a data do censo. Vamos obter uma lista dos estados exclusivos,
# então podemos iterar sobre todos os estados e para cada estado reduzimos o quadro de dados e calculamos o
# média.

# Vamos executar essa tarefa por 3 vezes e cronometrar. Para isso usaremos a função mágica da célula %%timeit

In [5]:
%%timeit -n 3

for state in df['STNAME'].unique():
    # vamos usar o numpy para calcular o average
    avg = np.average(df.where(df['STNAME']==state).dropna()['CENSUS2010POP'])
    # e vamos imprimir isto 
    print('Counties in state ' + state + 
          ' have an average population of ' + str(avg))

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

In [6]:
# Se você rolar até o final dessa saída, verá que leva um bom tempo para terminar.
# Agora vamos tentar outra abordagem usando groupby()

In [7]:
%%timeit -n 3

# Para este método, começamos dizendo aos pandas que estamos interessados ​​em agrupar por nome de estado, este é o "split"
for group, frame in df.groupby('STNAME'):
    
    # Você notará que há dois valores que definimos aqui. groupby() retorna uma tupla, onde o primeiro valor é o
    # valor da chave pela qual estávamos tentando agrupar, neste caso um nome de estado específico, e o segundo é
    # dataframe projetado que foi encontrado para esse grupo
    
    # Agora incluímos nossa lógica na etapa "apply", que é calcular uma média do censo2010pop
    avg = np.average(frame['CENSUS2010POP'])
    # e imprima os resultados
    print('Counties in state ' + group + 
          ' have an average population of ' + str(avg))
# E não precisamos nos preocupar com a etapa de combinação neste caso, porque toda a nossa transformação de dados é
# realmente imprimindo resultados.

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

In [8]:
#podemos ver que é bem mais rapido

In [9]:
# Agora, 99% do tempo, você usará group by em uma ou mais colunas. Mas você também pode fornecer uma função para
# agrupa e usa isso para segmentar seus dados.

# Este é um exemplo um pouco fabricado, mas digamos que você tenha um grande trabalho em lote com muito processamento e
# você deseja trabalhar em apenas um terço dos estados em um determinado momento. Poderíamos criar alguma função que
# retorna um número entre zero e dois com base no primeiro caractere do nome do estado. Então podemos dizer ao groupby
#  para usar esta função para dividir nosso quadro de dados. É importante notar que para fazer isso você precisa
# para definir o índice do quadro de dados como a coluna pela qual você deseja agrupar primeiro.

# Vamos criar uma nova função chamada set_batch_number e se a primeira letra do parâmetro for M maiúscula
# retornaremos um 0. Se for um Q maiúsculo retornaremos um 1 e caso contrário retornaremos um 2. Então passaremos
# esta função para o quadro de dados

df = df.set_index('STNAME')

def set_batch_number(item):
    if item[0]<'M':
        return 0
    if item[0]<'Q':
        return 1
    return 2


# O dataframe deve ser agrupado de acordo com o número do lote E faremos um loop em cada lote
# grupo
for group, frame in df.groupby(set_batch_number):
    print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')

There are 1177 records in group 0 for processing.
There are 1134 records in group 1 for processing.
There are 831 records in group 2 for processing.


In [10]:
# Observe que desta vez não passei um nome de coluna para groupby(). Em vez disso, defino o índice do dataframe
# seja STNAME, e se nenhum identificador de coluna for passado, groupby() usará automaticamente o índice.

In [12]:
# Vamos dar mais uma olhada em um exemplo de como podemos agrupar dados. Neste exemplo, quero usar um conjunto de dados
# de habitação do airbnb. Neste conjunto de dados existem duas colunas de interesse, uma é a cancel_policy
# e o outro é o review_scores_value.
df=pd.read_csv("/content/listings.csv")
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [13]:
# Então, como eu agruparia por ambas as colunas? Uma primeira abordagem pode ser promovê-los a um multiíndice
# e apenas chame groupby()
df=df.set_index(["cancellation_policy","review_scores_value"])

#agora que temos um multiinddex precisamos passar qual level de interesse usando groupby
for group, frame in df.groupby(level=(0,1)):
    print(group)

('flexible', 2.0)
('flexible', 4.0)
('flexible', 5.0)
('flexible', 6.0)
('flexible', 7.0)
('flexible', 8.0)
('flexible', 9.0)
('flexible', 10.0)
('moderate', 2.0)
('moderate', 4.0)
('moderate', 6.0)
('moderate', 7.0)
('moderate', 8.0)
('moderate', 9.0)
('moderate', 10.0)
('strict', 2.0)
('strict', 3.0)
('strict', 4.0)
('strict', 5.0)
('strict', 6.0)
('strict', 7.0)
('strict', 8.0)
('strict', 9.0)
('strict', 10.0)
('super_strict_30', 6.0)
('super_strict_30', 7.0)
('super_strict_30', 8.0)
('super_strict_30', 9.0)
('super_strict_30', 10.0)


In [14]:
# Isso parece funcionar bem. Mas e se quiséssemos agrupar pela política de cancelamento e revisar as pontuações, mas
# separar todos os 10 com menos de dez anos? Nesse caso, poderíamos usar uma função para gerenciar o
# agrupamentos
def grouping_fun(item):
   
    if item[1] == 10.0:
        return (item[0],"10.0")
    else:
        return (item[0],"not 10.0")

for group, frame in df.groupby(by=grouping_fun):
    print(group)

('flexible', '10.0')
('flexible', 'not 10.0')
('moderate', '10.0')
('moderate', 'not 10.0')
('strict', '10.0')
('strict', 'not 10.0')
('super_strict_30', '10.0')
('super_strict_30', 'not 10.0')


In [15]:
df.head()

id  \
cancellation_policy review_scores_value             
moderate            NaN                  12147973   
                    9.0                   3075044   
                    10.0                     6976   
                    10.0                  1436513   
flexible            10.0                  7651065   

                                                                   listing_url  \
cancellation_policy review_scores_value                                          
moderate            NaN                  https://www.airbnb.com/rooms/12147973   
                    9.0                   https://www.airbnb.com/rooms/3075044   
                    10.0                     https://www.airbnb.com/rooms/6976   
                    10.0                  https://www.airbnb.com/rooms/1436513   
flexible            10.0                  https://www.airbnb.com/rooms/7651065   

                                              scrape_id last_scraped  \
cancellation_policy review_scores_value                                
moderate            NaN                  20160906204935   2016-09-07   
                    9.0                  20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
flexible            10.0                 20160906204935   2016-09-07   

                                                                                  name  \
cancellation_policy review_scores_value                                                  
moderate            NaN                                     Sunny Bungalow in the City   
                    9.0                              Charming room in pet friendly apt   
                    10.0                              Mexican Folk Art Haven in Boston   
                    10.0                 Spacious Sunny Bedroom Suite in Historic Home   
flexible            10.0                                           Come Home to Boston   

                                                                                   summary  \
cancellation_policy review_scores_value                                                      
moderate            NaN                  Cozy, sunny, family home.  Master bedroom high...   
                    9.0                  Charming and quiet room in a second floor 1910...   
                    10.0                 Come stay with a friendly, middle-aged guy in ...   
                    10.0                 Come experience the comforts of home away from...   
flexible            10.0                 My comfy, clean and relaxing home is one block...   

                                                                                     space  \
cancellation_policy review_scores_value                                                      
moderate            NaN                  The house has an open and cozy feel at the sam...   
                    9.0                  Small but cozy and quite room with a full size...   
                    10.0                 Come stay with a friendly, middle-aged guy in ...   
                    10.0                 Most places you find in Boston are small howev...   
flexible            10.0                 Clean, attractive, private room, one block fro...   

                                                                               description  \
cancellation_policy review_scores_value                                                      
moderate            NaN                  Cozy, sunny, family home.  Master bedroom high...   
                    9.0                  Charming and quiet room in a second floor 1910...   
                    10.0                 Come stay with a friendly, middle-aged guy in ...   
                    10.0                 Come experience the comforts of home away from...   
flexible            10.0                 My comfy, clean and relaxing home is one 

### Até este ponto, aplicamos um processamento muito simples aos nossos dados após a divisão, apenas produzindo algumas instruções de impressão para demonstrar como a divisão funciona. Os desenvolvedores de pandas têm três amplos categorias de processamento de dados a acontecer durante a etapa de aplicação, **Agregação** de dados do grupo, **Transformação** de dados do grupo e **Filtragem** dos dados do grupo

# Aggregation

In [16]:
# A etapa de aplicação mais direta é a agregação de dados e usa o método agg() no groupby
#objeto. Até agora, apenas iteramos pelo objeto groupby, descompactando-o em um rótulo (o grupo
# nome) e um dataframe. Mas com agg podemos passar um dicionário das colunas que nos interessam
# agregando junto com a função que estamos procurando aplicar para agregar.

# Vamos redefinir o índice dos nossos dados do airbnb
df=df.reset_index()

# Agora vamos agrupar pela política de cancelamento e encontrar o review_scores_value médio por grupo
df.groupby("cancellation_policy").agg({"review_scores_value":np.average})

,review_scores_value
cancellation_policy,
flexible,NaN
moderate,NaN
strict,NaN
super_strict_30,NaN


In [18]:
#parece que nao funcionou, o problema esta na função que usamos para agregar ela nao ignora NaN
#No entanto, existe uma função que podemos usar para isso
df.groupby("cancellation_policy").agg({"review_scores_value":np.nanmean})

,review_scores_value
cancellation_policy,
flexible,9.237421
moderate,9.307398
strict,9.081441
super_strict_30,8.537313


In [19]:
# Podemos apenas estender este dicionário para agregar por múltiplas funções ou múltiplas colunas.
df.groupby("cancellation_policy").agg({"review_scores_value":(np.nanmean,np.nanstd),
                                      "reviews_per_month":np.nanmean})

review_scores_value           reviews_per_month
                                nanmean    nanstd           nanmean
cancellation_policy                                                
flexible                       9.237421  1.096271          1.829210
moderate                       9.307398  0.859859          2.391922
strict                         9.081441  1.040531          1.873467
super_strict_30                8.537313  0.840785          0.340143

In [ ]:
# Reserve um momento para ter certeza de que entendeu a célula anterior, pois ela é um pouco complexa. Primeiro estamos fazendo
# um grupo por no objeto dataframe pela coluna "cancellation_policy". Isso cria um novo objeto GroupBy.
# Então estamos invocando a função agg() nesse objeto. A função agg vai aplicar um ou mais
# funções que especificamos para os dataframes do grupo e retornam uma única linha por dataframe/grupo. Quando nós chamamos
# esta função enviamos duas entradas de dicionário, cada uma com a chave indicando qual coluna queríamos
# funções aplicadas. Para a primeira coluna, fornecemos uma tupla de duas funções. Observe que esses
# não são invocações de funções, como np.nanmean(), ou nomes de funções, como "nanmean", são referências a elas
# funções que retornarão valores únicos. O objeto groupby reconhecerá a tupla e chamará cada
# função em ordem na mesma coluna. Os resultados estarão em um índice hierárquico, mas como são
# colunas que eles não mostram como um índice em si. Então indicamos outra coluna e uma única função que queríamos.

#Transformation


In [20]:
# Transformação é diferente de agregação. Onde agg() retorna um único valor por coluna, então uma linha por
# group, tranform() retorna um objeto com o mesmo tamanho do grupo. Essencialmente, ele transmite a
# função que você fornece sobre o dataframe agrupado, retornando um novo dataframe. Isso torna a combinação de dados mais fácil.



In [22]:
# Por exemplo, suponha que queremos incluir os valores médios de classificação em um determinado grupo por política de cancelamento,
# mas preserva a forma do dataframe para que possamos gerar uma diferença entre uma observação individual
# e a soma.

# Primeiro, vamos definir apenas alguns subconjuntos de colunas em que estamos interessados
cols=['cancellation_policy','review_scores_value']
# Agora vamos transformá-lo, vou armazenar isso em seu próprio dataframe
transform_df=df[cols].groupby('cancellation_policy').transform(np.nanmean)
transform_df.head()

,review_scores_value
0,9.307398
1,9.307398
2,9.307398
3,9.307398
4,9.237421


In [23]:
# Assim, podemos ver que o índice aqui é realmente o mesmo que o dataframe original. Então vamos apenas juntar isso
# mas antes de fazermos isso, vamos renomear a coluna na versão transformada
transform_df.rename({'review_scores_value':'mean_review_scores'}, axis='columns', inplace=True)
df=df.merge(transform_df, left_index=True, right_index=True)
df.head()

,cancellation_policy,review_scores_value,id,listing_url,scrape_id,last_scraped,name,summary,space,description,...,review_scores_location,requires_license,license,jurisdiction_names,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores
0,moderate,NaN,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",...,NaN,f,NaN,NaN,f,f,f,1,NaN,9.307398
1,moderate,9.0,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,...,9.0,f,NaN,NaN,t,f,f,1,1.30,9.307398
2,moderate,10.0,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",...,9.0,f,NaN,NaN,f,t,f,1,0.47,9.307398
3,moderate,10.0,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,...,10.0,f,NaN,NaN,f,f,f,1,1.00,9.307398
4,flexible,10.0,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",...,9.0,f,NaN,NaN,f,f,f,1,2.25,9.237421


In [24]:
# Ótimo, podemos ver que nossa nova coluna está no lugar, a mean_review_scores. Então agora poderíamos criar, por
# instância, a diferença entre uma determinada linha e seu grupo (a política de cancelamento) significa.
df['mean_diff']=np.absolute(df['review_scores_value']-df['mean_review_scores'])
df['mean_diff'].head()

0         NaN
1    0.307398
2    0.692602
3    0.692602
4    0.762579
Name: mean_diff, dtype: float64

# Filtering


In [ ]:
# O objeto GroupBy também tem suporte embutido para filtrar grupos. Muitas vezes você vai querer agrupar
# por algum recurso, faça alguma transformação nos grupos e, em seguida, descarte certos grupos como parte de seu
# rotinas de limpeza. A função filter() recebe uma função que se aplica a cada dataframe de grupo e
# retorna True ou False, dependendo se esse grupo deve ser incluído nos resultados.


In [25]:
# Por exemplo, se queremos apenas os grupos que têm uma classificação média acima de 9 incluídos em nossos resultados
df.groupby('cancellation_policy').filter(lambda x: np.nanmean(x['review_scores_value'])>9.2)

,cancellation_policy,review_scores_value,id,listing_url,scrape_id,last_scraped,name,summary,space,description,...,requires_license,license,jurisdiction_names,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores,mean_diff
0,moderate,NaN,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",...,f,NaN,NaN,f,f,f,1,NaN,9.307398,NaN
1,moderate,9.0,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,...,f,NaN,NaN,t,f,f,1,1.30,9.307398,0.307398
2,moderate,10.0,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",...,f,NaN,NaN,f,t,f,1,0.47,9.307398,0.692602
3,moderate,10.0,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,...,f,NaN,NaN,f,f,f,1,1.00,9.307398,0.692602
4,flexible,10.0,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",...,f,NaN,NaN,f,f,f,1,2.25,9.237421,0.762579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3576,flexible,NaN,14689681,https://www.airbnb.com/rooms/14689681,20160906204935,2016-09-07,Beautiful loft style bedroom with large bathroom,You'd be living on the top floor of a four sto...,NaN,You'd be living on the top floor of a four sto...,...,f,NaN,NaN,f,f,f,1,NaN,9.237421,NaN
3577,flexible,NaN,13750763,https://www.airbnb.com/rooms/13750763,20160906204935,2016-09-07,Comfortable Space in the Heart of Brookline,"Our place is close to Coolidge Corner, Allston...",This space consists of 2 Rooms and a private b...,"Our place is close to Coolidge Corner, Allston...",...,f,NaN,NaN,f,f,f,1,NaN,9.237421,NaN
3579,flexible,NaN,14852179,https://www.airbnb.com/rooms/14852179,20160906204935,2016-09-07,Spacious Queen Bed Room Close to Boston Univer...,- Grocery: A full-size Star market is 2 minute...,NaN,- Grocery: A full-size Star market is 2 minute...,...,f,NaN,NaN,f,f,f,1,NaN,9.237421,NaN
3582,flexible,NaN,14585486,https://www.airbnb.com/rooms/14585486,20160906204935,2016-09-07,Gorgeous funky apartment,Funky little apartment close to public transpo...,Modern and relaxed space with many facilities ...,Funky little apartment close to public transpo...,...,f,NaN,NaN,f,f,f,1,NaN,9.237421,NaN


In [ ]:
# Observe que os resultados ainda estão indexados, mas que qualquer um dos resultados que estavam em um grupo com média
# pontuação de revisão menor ou igual a 9,2 não foi copiada.

# Applying

In [26]:
# De longe, a operação mais comum que invoco em objetos groupby é a função apply(). Isso permite que você
# aplica uma função arbitrária a cada grupo e costura os resultados de volta para cada apply() em um único
# dataframe onde o índice é preservado.

# Vamos ver um exemplo usando nossos dados do airbnb, vou pegar uma cópia limpa do dataframe

df=pd.read_csv("/content/listings.csv")

# vamos separar previamente as colunas de interesse 
df=df[['cancellation_policy','review_scores_value']]
df.head()

,cancellation_policy,review_scores_value
0,moderate,NaN
1,moderate,9.0
2,moderate,10.0
3,moderate,10.0
4,flexible,10.0


In [27]:
# No trabalho anterior, queríamos encontrar a pontuação média de revisão de uma lista e seu desvio do grupo
# significa. Este foi um processo de duas etapas, primeiro usamos transform() no objeto groupby e depois tivemos 
# transmitir para criar uma nova coluna. Com apply() podemos agrupar essa lógica em um só lugar
def calc_mean_review_scores(group):
    
    # group é um dataframe apenas do que agrupamos, por exemplo política de cancelamento, para que possamos tratar
    # isso como o dataframe completo
    avg=np.nanmean(group["review_scores_value"])
    # agora transmita nossa fórmula e crie uma nova coluna
    group["review_scores_mean"]=np.abs(avg-group["review_scores_value"])
    return group

# Agora é só aplicar isso aos grupos
df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()


,cancellation_policy,review_scores_value,review_scores_mean
0,moderate,NaN,NaN
1,moderate,9.0,0.307398
2,moderate,10.0,0.692602
3,moderate,10.0,0.692602
4,flexible,10.0,0.762579
